In [1]:
!pip install -r requirements.txt

  Using cached attrs-20.2.0-py2.py3-none-any.whl (48 kB)
     |████████████████████████████████| 230 kB 365 kB/s eta 0:00:01
  Using cached dill-0.3.2.zip (177 kB)
  Using cached iniconfig-1.0.1-py3-none-any.whl (4.2 kB)
  Using cached ipython-7.18.1-py3-none-any.whl (786 kB)
  Using cached isort-5.5.4-py3-none-any.whl (95 kB)
  Using cached jedi-0.17.2-py2.py3-none-any.whl (1.4 MB)
  Using cached jupyterthemes-0.20.0-py2.py3-none-any.whl (7.0 MB)
     |████████████████████████████████| 92 kB 19 kB/s  eta 0:00:01
  Using cached lesscpy-0.14.0-py2.py3-none-any.whl (46 kB)
  Using cached multiprocess-0.70.10.zip (2.4 MB)
  Using cached nbclient-0.5.0-py3-none-any.whl (65 kB)
  Using cached nbformat-5.0.7-py3-none-any.whl (170 kB)
  Using cached nest_asyncio-1.4.1-py3-none-any.whl (5.2 kB)
  Using cached pandocfilters-1.4.2.tar.gz (14 kB)
  Using cached parso-0.7.1-py2.py3-none-any.whl (109 kB)
  Using cached pathos-0.2.6.zip (219 kB)
     |████████████████████████████████| 2.2 MB 172 kB/

  Using cached prompt_toolkit-3.0.7-py3-none-any.whl (355 kB)
  Using cached Pygments-2.7.1-py3-none-any.whl (944 kB)
  Using cached pytest-6.1.1-py3-none-any.whl (272 kB)
     |████████████████████████████████| 98 kB 175 kB/s eta 0:00:01
  Created wheel for dill: filename=dill-0.3.2-py3-none-any.whl size=78912 sha256=8cc879c3cbb3858e846d2b2d68fb579b186c4adcc54c3d477c0e6cf885745fde
  Stored in directory: /home/vitor/.cache/pip/wheels/93/7f/7d/78ec535a4340ef2696aad8b17fe8bb063d56301bd62881b069
  Created wheel for multiprocess: filename=multiprocess-0.70.10-py3-none-any.whl size=125838 sha256=d84ff5e33c066f62f758083668d3a482f2b93fe7f2d962b88bc2bf5de342b069
  Stored in directory: /home/vitor/.cache/pip/wheels/f4/c7/82/0bcdc3506e6a8365963db24f477b81693438d4b9165c0757b7
  Created wheel for pandocfilters: filename=pandocfilters-1.4.2-py3-none-any.whl size=7855 sha256=8e514bd326400ad4674f81973fd5aab5cbc11da1f6b168c1d9ff02e2212cf9a2
  Stored in directory: /home/vitor/.cache/pip/wheels/f6/08/65

    Uninstalling jedi-0.17.1:
      Successfully uninstalled jedi-0.17.1
  Attempting uninstall: ipython
    Found existing installation: ipython 7.19.0
    Uninstalling ipython-7.19.0:
      Successfully uninstalled ipython-7.19.0
  Attempting uninstall: isort
    Found existing installation: isort 5.6.4
    Uninstalling isort-5.6.4:
      Successfully uninstalled isort-5.6.4
  Attempting uninstall: kiwisolver
    Found existing installation: kiwisolver 1.3.0
    Uninstalling kiwisolver-1.3.0:
      Successfully uninstalled kiwisolver-1.3.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.11.1
    Uninstalling multiprocess-0.70.11.1:
      Successfully uninstalled multiprocess-0.70.11.1
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.4.2
    Uninstalling nest-asyncio-1.4.2:
      Successfully uninstalled nest-asyncio-1.4.2
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.0.8
    Uni

In [2]:
#!/usr/bin/env python
# coding: utf-8

import argparse
import json
from network_utils import get_participants, get_proposals

import pandas as pd
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionContext, ExecutionMode, Executor
from cadCAD import configs

from entities import ProposalStatus
from simulation import (CommonsSimulationConfiguration, bootstrap_simulation,
                        partial_state_update_blocks)
from utils import new_random_number_func


def get_simulation_metrics(df_final):
    average_sentiment = 0


def run_simulation(c: CommonsSimulationConfiguration):
    initial_conditions, simulation_parameters = bootstrap_simulation(c)

    exp = Experiment()
    exp.append_configs(
        initial_state=initial_conditions,
        partial_state_update_blocks=partial_state_update_blocks,
        sim_configs=simulation_parameters
    )

    # Do not use multi_proc, breaks ipdb.set_trace()
    exec_mode = ExecutionMode()
    single_proc_context = ExecutionContext(exec_mode.local_mode)
    executor = Executor(single_proc_context, configs)

    raw_system_events, tensor_field, sessions = executor.execute()

    df = pd.DataFrame(raw_system_events)
    return df


def get_simulation_results(c):
    df = run_simulation(c)
    df_final = df[df.substep.eq(2)]
    random_func = new_random_number_func(None)

    last_network = df_final.iloc[-1, 0]
    candidates = len(get_proposals(last_network, status=ProposalStatus.CANDIDATE))
    actives = len(get_proposals(last_network, status=ProposalStatus.ACTIVE))
    completed = len(get_proposals(last_network, status=ProposalStatus.COMPLETED))
    failed = len(get_proposals(last_network, status=ProposalStatus.FAILED))
    participants = len(get_participants(last_network))

    result = {
        "timestep": list(df_final["timestep"]),
        "funding_pool": list(df_final["funding_pool"]),
        "token_price": list(df_final["token_price"]),
        "sentiment": list(df_final["sentiment"]),
        "score": int(random_func() * 1000),
        "participants": participants,
        "proposals": {
            "candidates": candidates,
            "actives": actives,
            "completed": completed,
            "failed": failed
        }
    }
    return result, df_final




In [3]:
hatchers_count = 35

In [4]:
c = CommonsSimulationConfiguration(timesteps_days=200, days_to_80p_of_max_voting_weight=15, hatchers=hatchers_count, proposals=35, exit_tribute=0.5)
result, df_final = get_simulation_results(c)

COLLATERAL 0.16

                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (200, 9, 1, 8)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 147.07s


In [5]:
df_final

,network,commons,funding_pool,collateral_pool,token_supply,token_price,policy_output,sentiment,simulation,subset,run,substep,timestep
2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f653b13c370>,3.182451e+06,1.272980e+07,1.591226e+08,0.160000,None,0.750000,0,0,1,2,1
21,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f653a7426a0>,3.183241e+06,1.273084e+07,1.591290e+08,0.160007,"{'succeeded': [], 'identity': 0, 'failed': []}",0.765708,0,0,1,2,2
40,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f653a837580>,3.183998e+06,1.273396e+07,1.591485e+08,0.160026,"{'succeeded': [], 'identity': 0, 'failed': []}",0.760708,0,0,1,2,3
59,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f65394bed00>,3.184988e+06,1.273598e+07,1.591611e+08,0.160039,"{'succeeded': [], 'identity': 0, 'failed': []}",0.755708,0,0,1,2,4
78,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f6538a028b0>,3.185799e+06,1.273973e+07,1.591846e+08,0.160062,"{'succeeded': [], 'identity': 0, 'failed': []}",0.750708,0,0,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3707,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f647f16ce80>,4.499323e+06,1.098934e+06,4.675270e+07,0.047011,"{'succeeded': [58], 'identity': 0, 'failed': [...",0.686419,0,0,1,2,196
3726,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f647dce0eb0>,4.502425e+06,1.097311e+06,4.671816e+07,0.046976,"{'succeeded': [], 'identity': 0, 'failed': []}",0.681419,0,0,1,2,197
3745,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f647c942c70>,4.503714e+06,1.098441e+06,4.674220e+07,0.047000,"{'succeeded': [113], 'identity': 0, 'failed': []}",0.715411,0,0,1,2,198
3764,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",<hatch.Commons object at 0x7f647b534f10>,4.290664e+06,1.058959e+06,4.589448e+07,0.046148,"{'succeeded': [120], 'identity': 0, 'failed': []}",0.759934,0,0,1,2,199


# Metrics

In [6]:
scores = []

## 1 -  price compared to hatch price

In [7]:
hatch_price = df_final.iloc[0, 5]
final_price = df_final.iloc[-1, 5]
ratio_price_1 = final_price/hatch_price
scores.append(ratio_price_1)
print("PRICE RATIO:", ratio_price_1)

PRICE RATIO: 0.28838661120662323


## 2 - Sentiment at the end of the 3 years

In [8]:
final_sentiment_2 = df_final.iloc[-1, 7]
scores.append(final_sentiment_2)
print("FINAL SENTIMENT:", final_sentiment_2)

FINAL SENTIMENT: 0.7549337444576248


## 3 - Number of proposals funded compared to initial proposals

In [9]:
last_network = df_final.iloc[-1, 0]
all_proposals = len(get_proposals(last_network))
candidates = len(get_proposals(last_network, status=ProposalStatus.CANDIDATE))

funded = all_proposals - candidates
initial_proposals = len(get_proposals(df_final.iloc[0, 0], status=ProposalStatus.CANDIDATE))
ratio_funded_3 = funded/initial_proposals
scores.append(ratio_funded_3)
print("RATIO FUNDED BY INITIAL PROPOSALS:", ratio_funded_3)

RATIO FUNDED BY INITIAL PROPOSALS: 1.8571428571428572


## 4 - Total spent by the funding pool compared to the amount received in the hatch phase

In [10]:
proposals = get_proposals(last_network)

total_spent = 0
for _, proposal in proposals:
    if proposal.status == ProposalStatus.CANDIDATE:
        pass
    else: 
        total_spent += proposal.funds_requested
        
funding_pool_hatch = df_final.iloc[0, 2]

print("FUNDS RECEIVED ON HACTH PHASE:",funding_pool_hatch)     
print("TOTAL SPENT BY THE FUNDING POOL:", total_spent)
ratio_spent_4 = total_spent/funding_pool_hatch
scores.append(ratio_spent_4)
print("RATIO:", ratio_spent_4)

FUNDS RECEIVED ON HACTH PHASE: 3182451.0144401076
TOTAL SPENT BY THE FUNDING POOL: 5177607.660194323
RATIO: 1.6269245423421628


## 5 - Average amount in funding pool over time compared to the amount received in the hatch phase

In [11]:
avg_funding_pool = df_final["funding_pool"].mean()
funding_pool_hatch = df_final.iloc[0, 2]
avg_funding_pool_ratio_5 = avg_funding_pool/funding_pool_hatch
scores.append(avg_funding_pool_ratio_5)
print("RATIO:", avg_funding_pool_ratio_5)

RATIO: 1.8116791984677547


## 6 - Average price compared with hatch price

In [12]:
avg_price = df_final["token_price"].mean()
hatch_price = df_final.iloc[0, 5]
print("AVG PRICE:", avg_price, "HATCH PRICE:", hatch_price)
avg_price_ratio_6 = avg_price/hatch_price
scores.append(avg_price_ratio_6)
print("PRICE RATIO:", avg_price_ratio_6)

AVG PRICE: 0.09491751996705645 HATCH PRICE: 0.16
PRICE RATIO: 0.5932344997941028


## 7 - Average sentiment

In [13]:
avg_sentiment_7 = df_final["sentiment"].mean()
scores.append(avg_sentiment_7)
print("AVERAGE SENTIMENT:", avg_sentiment_7)

AVERAGE SENTIMENT: 0.5850999842418536


## 8 - Ratio of successful projects to failed ones

In [14]:
completed = len(get_proposals(last_network, status=ProposalStatus.COMPLETED))
failed = len(get_proposals(last_network, status=ProposalStatus.FAILED))

print("COMPLETED:", completed, "FAILED:", failed)
ratio_completed_failed_8 = completed/failed
scores.append(ratio_completed_failed_8)
print("RATIO completed/failed", ratio_completed_failed_8)

COMPLETED: 30 FAILED: 31
RATIO completed/failed 0.967741935483871


## 9 - Nº of final participants compared to Nº of hatchers

In [15]:
last_network = df_final.iloc[-1, 0]

participants = len(get_participants(last_network))
print("PARTICIPANTS:",participants)
ratio_participants_9 = participants/hatchers_count
scores.append(ratio_participants_9)
print("RATIO hatchers/partcicipants", ratio_participants_9)

PARTICIPANTS: 44
RATIO hatchers/partcicipants 1.2571428571428571


# Final Score

In [16]:
print(scores)
print("FINAL SCORE", sum(scores)*100)

[0.28838661120662323, 0.7549337444576248, 1.8571428571428572, 1.6269245423421628, 1.8116791984677547, 0.5932344997941028, 0.5850999842418536, 0.967741935483871, 1.2571428571428571]
FINAL SCORE 974.2286230279707
